In [24]:
import sys
sys.path.append('diffusion/stable_diffusion/')
sys.path.append('diffusion/stable_diffusion/model/')
sys.path.append('lib/src/')
import torch
import os
import numpy as np

from diffusion.stable_diffusion.latent_diffusion import LatentDiffusion
from diffusion.stable_diffusion.model.autoencoder import Autoencoder
from diffusion.stable_diffusion.model.clip_embedder import CLIPTextEmbedder
from diffusion.stable_diffusion.model.unet import UNetModel




from lib.src.pythae.models import LVAE_IAF, LVAE_IAF_Config, AutoModel
from lib.src.pythae.models.nn.base_architectures import BaseDecoder
from lib.scripts.utils import Encoder_Chairs,Decoder_Chairs, My_Dataset, My_MaskedDataset, make_batched_masks

def load_config_unet(config):
    return UNetModel(
        in_channels=config['in_channels'],
        out_channels=config['out_channels'],
        channels=config['channels'],
        n_res_blocks=config['n_res_blocks'],
        attention_levels=config['attention_levels'],
        channel_multipliers=config['channel_multipliers'],
        n_heads=config['n_heads'],
    )

%reload_ext autoreload
%autoreload 2

In [5]:
train_data = torch.load(os.path.join('lib/my_data/sprites/Sprites_train.pt'))[:-1000]
eval_data = torch.load(os.path.join('lib/my_data/sprites/Sprites_train.pt'), map_location="cpu")[-1000:]
test_data = torch.load(os.path.join('lib/my_data/sprites/Sprites_test.pt'), map_location="cpu")

In [6]:
input_dim = (3, 64, 64)
latent_dim = 16
beta = 1
n_hidden_in_made = 3
n_made_blocks = 2
warmup = 10
context_dim = None
prior = 'vamp'
posterior = 'iaf'
vamp_number_components= 500
linear_scheduling_steps = 10
num_epochs = 200
batch_size = 64
learning_rate=  1e-3 
steps_saving = None
steps_predict = 100
shuffle_data = True

config = LVAE_IAF_Config(
    input_dim=input_dim,
    n_obs_per_ind=8,
    latent_dim=latent_dim,
    beta=beta,
    n_hidden_in_made=n_hidden_in_made,
    n_made_blocks=n_made_blocks,
    warmup=warmup,
    context_dim=context_dim,
    prior=prior,
    posterior=posterior,
    vamp_number_components=vamp_number_components,
    linear_scheduling_steps=linear_scheduling_steps
)


encoder = Encoder_Chairs(config)
decoder = Decoder_Chairs(config)
vae = LVAE_IAF(config, encoder, decoder)

vae.load_state_dict(torch.load('lib/dummy_output_dir/LVAE_IAF_training_2024-05-01_17-22-06/final_model/model.pt')['model_state_dict'])


<All keys matched successfully>

In [124]:
z = torch.randn(1, 3, 256, 256)
conv = torch.nn.Conv2d(in_channels=3, out_channels=64, kernel_size=5, padding=2)

print(conv(z).shape)
print(sum(p.numel() for p in conv.parameters() if p.requires_grad))

torch.Size([1, 64, 256, 256])
4864


In [29]:
########
in_channels = 3
out_channels = 1
channels = 64
n_res_blocks = 2
attention_levels = [6]
channel_multipliers = (1, 2)
n_heads = 2

unet_config = {
    'in_channels': in_channels,
    'out_channels': out_channels,
    'channels': channels,
    'n_res_blocks': n_res_blocks,
    'attention_levels': attention_levels,
    'channel_multipliers': channel_multipliers,
    'n_heads': n_heads,
}

unet = load_config_unet(unet_config)
z = torch.randn(1, 3, 256, 256)
empty_prompt_embed = None
unet(z, torch.tensor([10]), empty_prompt_embed).shape

d_time_emb 256
Channels list [64, 128]
0
1
torch.Size([1, 64, 256, 256])
torch.Size([1, 64, 256, 256])
torch.Size([1, 64, 256, 256])
torch.Size([1, 64, 128, 128])
torch.Size([1, 128, 128, 128])
torch.Size([1, 128, 128, 128])
Entering middle block  torch.Size([1, 128, 128, 128])
Shap entering the trans:  torch.Size([1, 128, 128, 128])
Middle block done  torch.Size([1, 128, 128, 128])


torch.Size([1, 1, 256, 256])

In [79]:
from diffusion.stable_diffusion.model.clip_embedder import CLIPTextEmbedder

clip = CLIPTextEmbedder(device = 'cuda')
prompt = ''
empty_prompt_embed = clip(prompt)
empty_prompt_embed = empty_prompt_embed.to('cpu')

torch.Size([1, 77])
